In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from itertools import combinations
from collections import Counter, defaultdict
from astropy.table import Table, join, vstack, unique
from astropy.coordinates import SkyCoord, Distance
from SAGA.utils import add_skycoord, find_near_objid, find_near_ra_dec, find_objid
from easyquery import Query, QueryMaker

import SAGA
from SAGA.database import FitsTable, GoogleSheets
from SAGA import ObjectCuts as C
from SAGA.utils import add_skycoord
from SAGA.utils.distance import z2v, d2m, m2d
from SAGA.utils.display import *
print(SAGA.__version__)

In [ ]:
objects = np.array([5618260000000001312, 3125140000000000628, 5597470000000002640, 4456920000000002940, 4427340000000003054, 3597300000000002624, 4251140000000000989, 5263260000000002113], np.int).reshape(4, 2).T

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Downloads/saga_base_all.fits").read()
base = QueryMaker.in1d("OBJID", objects.flatten()).filter(base)
assert len(base) == 8

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import os

params_dict = dict(pixscale=0.2, size=200, layer="dr8")
base_url = "http://legacysurvey.org/viewer-dev/jpeg-cutout/"
cache_dir = "/home/yymao/Downloads/"

def load_image(*args, **kwargs):
    response = requests.get(*args, **kwargs)
    return response.content

def get_satellite_image(sat):
    
    fname = f"{sat['survey']}-{sat['OBJID']}.jpg"
    fpath = os.path.join(cache_dir, fname)
    try:
        img = open(fpath, "rb").read()
    except FileNotFoundError:    
        params_dict["ra"] = sat["RA"]
        params_dict["dec"] = sat["DEC"]
        params_dict["layer"] = "des-dr1" if sat["OBJID"] == 467510764 else "dr8"
        img = load_image(base_url, params_dict)
        open(fpath, "wb").write(img)
    return Image.open(BytesIO(img))

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(8, 4.))

for ax_this, obj_this in zip(ax.flat, objects.flat):
    sat = find_objid(base, obj_this)
    ax_this.imshow(get_satellite_image(sat))
    ax_this.set_axis_off()
    ax_this.text(10, 10, ("$z={:.4f}$".format(sat["SPEC_Z"]) if sat["ZQUALITY"] >= 3 else "$z=?$"), va="top", fontsize=13, color="w")
    ax_this.text(10, 190, "$r_o={:.1f},\\;\\mu_\\mathrm{{eff}}={:.1f}$".format(sat["r_mag"], sat["sb_r"]), va="bottom", fontsize=13, color="w")

ax[0][0].set_title("Satellites")
ax[0][1].set_title("Low-$z$ dLSBGs")
ax[0][2].set_title("dLSBGs, w/o $z$")
ax[0][3].set_title("Non-diffused LSBG")

fig.tight_layout(pad=0, h_pad=0, w_pad=0)
plt.savefig("/home/yymao/Downloads/lsbg-example.pdf", bbox_inches="tight")